# Import

In [1]:
# import moduels
import h5py
import numpy as np
import matplotlib.pylab as plt
import math
from PIL import Image

# settings

In [2]:
# access the data
# this part have to be replaced when the images from the neuronal networks are available
dset = h5py.File("dataset_test.h5","r")

#Access to the input data
RGB = dset["RGB"]
NIR = dset["NIR"]
print("Thes shape of the images is:")
print(RGB.shape)


# get size and numbers of the dataset
# number of images
n_images = RGB.shape[0]

# imagesize x_direction
sx_image = RGB.shape[1]
#imageszize y_direction
sy_image = RGB.shape[2]

Thes shape of the images is:
(3, 10980, 10980, 3)


In [3]:
# set number of images in x and y direction

# number of images in x direction
nx = 5
#number of images in y direction 
ny = 5

# size of patch
# alle sind quadratisch
s_patch = 256

# numbers of categories in the pretrained network Pascal Voc --> 21
n_cat = 21

In [4]:
# compute the coordinates of all top left pixels of each patch
# is for each image the same

x_max = sx_image-s_patch
y_max = sy_image-s_patch 

stepsize_x = math.floor(x_max/nx)
stepsize_y = math.floor(y_max/ny)

#create arrays with the corrdinates
x_coordinates = np.arange(0,x_max,stepsize_x)
y_coordinates = np.arange(0,y_max,stepsize_y)

# create grid
xx , yy = np.meshgrid(x_coordinates,y_coordinates)

# Network part

In [5]:
# import networks modules
from torchvision import models
import torchvision.transforms as T

In [6]:
# select model
# available more models are avialable at: https://pytorch.org/vision/stable/models.html

#FCN ResNet50
model = models.segmentation.fcn_resnet50(pretrained=True, progress=True)

#DeepLabV3 ResNet50
#model = models.segmentation.deeplabv3_resnet50(pretrained=True, progress=True)

In [7]:
# function to prepare the input image into the correct form (for all segementation models of torchvision the same normalistation and standard deviation can be used!)
# more information: https://learnopencv.com/pytorch-for-beginners-image-classification-using-pre-trained-models/
# or https://pytorch.org/vision/stable/models.html
trf = T.Compose([T.Resize(s_patch),
                 T.CenterCrop(s_patch),
                 T.ToTensor(), 
                 T.Normalize(mean = [0.485, 0.456, 0.406], 
                             std = [0.229, 0.224, 0.225])])

In [9]:
# run over each image patch and store it

for i in range(len(yy)):
    for j in range(len(xx)):
        # Achtung muss hier noch angepasst werden um über alle Bilder zu iterien --> einfachster Weg for loop
        temp_img = Image.fromarray(RGB[1,xx[i][j]:xx[i][j]+s_patch, yy[i][j]:yy[i][j]+s_patch,:3])
        inp = trf(temp_img).unsqueeze(0) #check what unsqueeze does
        out = model(inp)['out']
        
        # reshape the output of the network to use it in the classifier 
        # size (image_size x image_size) x 21 (all of the torchvision segmentation models are trained with the Pascal Voc dataseet --> contains 21 categories) more info: https://pytorch.org/vision/stable/models.html
        
        # TODO check if the reshape is done in the correct way
        temp = out.detach().numpy()
        temp1 = temp[0]
        temp2 = temp1.reshape(np.square(s_patch),n_cat)
        
        # save array
        np.save('./output_network/'+str(xx[i][j])+'_'+str(yy[i][j])+'.npy',temp2)

In [ ]:
# test = np.load('test_ndarray.npy')